In [1]:
import pandas as pd
import numpy as np
import os
from sktime.annotation.eagglo import EAgglo
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
# sys.path.appendb
# List all CSV files in the data/raw directory
csv_files = [f for f in os.listdir('../data/raw') if f.endswith('.csv')]

# ,date,sender_id,bgl,bgl_date_millis,text,template,msg_type,affects_fob,affects_iob,dose_units,food_g,food_glycemic_index,dose_automatic,fp_bgl,message_basal_change,__typename,trend
# Define the columns to load
columns_to_load = ['date', 'bgl', 'msg_type', 'affects_fob', 'affects_iob','dose_units','food_g','food_glycemic_index']

# Load each CSV file into a DataFrame and store them in a dictionary
dataframes = {file: pd.read_csv(os.path.join('../data/raw', file), usecols=columns_to_load, parse_dates=['date']) for file in csv_files}

# Print the names of the loaded DataFrames
print("Loaded DataFrames:", list(dataframes.keys()))

Loaded DataFrames: ['500030_2024-07-01_2024-09-30.csv', '679372_2024-07-01_2024-09-30.csv']


In [2]:
df = dataframes['500030_2024-07-01_2024-09-30.csv']
df = df.head(5000)
scaler_bgl = StandardScaler()  # For blood glucose levels
scaler_food_g = MinMaxScaler()  # For food intake
scaler_food_gi = MinMaxScaler()  # For food glycemic index

df['bgl_scaled'] = scaler_bgl.fit_transform(df[['bgl']])
df['food_g_scaled'] = scaler_food_g.fit_transform(df[['food_g']])
df['food_gi_scaled'] = scaler_food_gi.fit_transform(df[['food_glycemic_index']])

/var/folders/yc/fj0w5k890gjbcm5p5bkv9vs40000gn/T/ipykernel_72627/3224705176.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['bgl_scaled'] = scaler_bgl.fit_transform(df[['bgl']])
/var/folders/yc/fj0w5k890gjbcm5p5bkv9vs40000gn/T/ipykernel_72627/3224705176.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['food_g_scaled'] = scaler_food_g.fit_transform(df[['food_g']])
/var/folders/yc/fj0w5k890gjbcm5p5bkv9vs40000gn/T/ipykernel_72627/3224705176.py:9: SettingWithCopyWarning: 
A value is trying to be se

In [4]:
columns_to_use = ['bgl_scaled', 'food_g_scaled', 'food_gi_scaled']
X = df[columns_to_use]
model = EAgglo(penalty="mean_diff_penalty")
cluster = model.fit_transform(X)
df['cluster'] = cluster


KeyboardInterrupt: 

In [39]:
df['cluster'].unique()

array([   0,    1,    2, ..., 4997, 4998, 4999])

In [38]:
# Assuming df['cluster'] contains cluster assignments and df['date'] is the time axis
plt.figure(figsize=(10, 6))

# Plot blood glucose levels (bgl) over time
plt.plot(df['date'], df['bgl'], label='Blood Glucose (BGL)', color='blue')

# Optionally plot other relevant variables, like food intake
plt.plot(df['date'], df['food_g'], label='Food Intake (g)', color='green', linestyle='dashed')

# Add shading or vertical lines to indicate clusters (identified segments)
cluster_colors = np.random.rand(df['cluster'].nunique(), 3)  # Generate random colors for clusters
for cluster in df['cluster'].unique():
    cluster_indices = df[df['cluster'] == cluster].index
    plt.axvspan(df['date'].iloc[cluster_indices[0]], df['date'].iloc[cluster_indices[-1]], 
                color=cluster_colors[cluster], alpha=0.2, label=f'Cluster {cluster}')

# Add labels and legend
plt.xlabel('Date')
plt.ylabel('BGL / Food Intake')
plt.title('Meal Identification: Blood Glucose and Food Intake over Time')
plt.legend()
plt.show()

ValueError: Image size of 964x104817 pixels is too large. It must be less than 2^16 in each direction.

<Figure size 1000x600 with 1 Axes>

In [ ]:
# Plot 1: Line Plot with Cluster Annotations

import matplotlib.pyplot as plt
import numpy as np

# Assuming df['cluster'] contains cluster assignments and df['date'] is the time axis
plt.figure(figsize=(10, 6))

# Plot blood glucose levels (bgl) over time
plt.plot(df['date'], df['bgl'], label='Blood Glucose (BGL)', color='blue')

# Optionally plot other relevant variables, like food intake
plt.plot(df['date'], df['food_g'], label='Food Intake (g)', color='green', linestyle='dashed')

# Add shading or vertical lines to indicate clusters (identified segments)
cluster_colors = np.random.rand(df['cluster'].nunique(), 3)  # Generate random colors for clusters
for cluster in df['cluster'].unique():
    cluster_indices = df[df['cluster'] == cluster].index
    plt.axvspan(df['date'].iloc[cluster_indices[0]], df['date'].iloc[cluster_indices[-1]], 
                color=cluster_colors[cluster], alpha=0.2, label=f'Cluster {cluster}')

# Add labels and legend
plt.xlabel('Date')
plt.ylabel('BGL / Food Intake')
plt.title('Meal Identification: Blood Glucose and Food Intake over Time')
plt.legend()
plt.show()
